# Eingabemaske

In [ ]:
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
from sklearn.neighbors import BallTree
import ipywidgets as widgets
from IPython.display import display
import folium
import geopandas as gpd
from shapely.geometry import Point
import joblib
from IPython.display import clear_output
import matplotlib.pyplot as plt
from pathlib import Path

# Pfad zu den JSON-Dateien
current_directory = Path().resolve()
path_to_data = current_directory / "Final_Ref_Data"

#path_to_data = 'C:/Users/Joel/Desktop/FH-Wedel/Learning und Softcomputing/Finally_XG Boost/Taining_Data_Calc/'

# Erstellen des BallTree
#coords = np.radians(gdf[['Latitude', 'Longitude']].values)
#tree = BallTree(coords, metric='haversine')
model = load_model('rent_per_sqm_model_nn.h5')
scaler = joblib.load('scaler1.pkl') 

def geocode_address(address):
    geolocator = Nominatim(user_agent="geoapiExercises")
    location = geolocator.geocode(address)
    return (location.latitude, location.longitude) if location else None

def find_nearby(point, gdf, tree, num_points=20):
    point = np.radians([point])
    dist, ind = tree.query(point, k=num_points)
    nearest_points = gdf.iloc[ind[0]]

    desc = nearest_points['quadratmeterpreis'].describe().drop(['25%', '50%', '75%'])
    desc.index = ['count_nearby', 'mean_rent_per_sqm_nearby', 'std_rent_per_sqm_nearby', 
              'min_rent_per_sqm_nearby', 'max_rent_per_sqm_nearby']
    
    for feature in ["hasBasement", "hasBalcony", "hasGarden", "hasElevator"]:
        desc[f'mean_{feature}_nearby'] = nearest_points[feature].mean()

    return desc, nearest_points[['Latitude', 'Longitude']]


def convert_house_type(house_type):
    house_type_mapping = {
        "Apartment": 0,
        "Groundfloor": 1,
        "half basement": 2,
        "roof storey": 3,
        "maisonette": 4,
        "raised ground floor": 5,
        "terraced flat": 6,
        "penthouse": 8,
        "loft": 9,
        "other": 7
    }

    return house_type_mapping.get(house_type, 0)  # Standardwert 0, falls keine Übereinstimmung gefunden wird

def predict(address, living_space, room_count, property_age, has_basement, has_balcony, parking_lot_count, has_garden, has_elevator, house_type, PredictedYear):
    # Laden des Datensatzes basierend auf dem ausgewählten Jahr
    data_path = path_to_data.joinpath(f'trainingData_fut_{PredictedYear}_calc_res.json')
    data = pd.read_json(data_path)
    data['quadratmeterpreis'] = data['rent'] / data['livingSpace']
    gdf = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data.Longitude, data.Latitude))

    # Erstellen des BallTree
    coords = np.radians(gdf[['Latitude', 'Longitude']].values)
    tree = BallTree(coords, metric='haversine')

    coordinates = geocode_address(address)

    if coordinates:
        nearby_features, reference_points = find_nearby(coordinates, gdf, tree)

        input_data = {
            'roomCount': room_count,
            'propertyAge': property_age,
            'livingSpace': living_space,
            'hasBasement': has_basement,
            'hasBalcony': has_balcony,
            'parkingLotCount': parking_lot_count,
            'hasGarden': has_garden,
            'hasElevator': has_elevator,
            'houseType': convert_house_type(house_type),
            **nearby_features.to_dict()}

        input_df = pd.DataFrame(input_data, index=[0])
        input_scaled = scaler.transform(input_df)
        prediction = model.predict(input_scaled)

        return prediction[0][0], input_df, coordinates, reference_points

    else:
        print('Unable to geocode address.')
        return None, None, None, None



from IPython.display import clear_output

def predict_button_clicked(b):
    clear_output(wait=True)
    
    address = address_widget.value
    living_space = living_space_widget.value
    room_count = room_count_widget.value
    property_age = property_age_widget.value
    has_basement = has_basement_widget.value
    has_balcony = has_balcony_widget.value
    parking_lot_count = parking_lot_count_widget.value
    has_garden = has_garden_widget.value
    has_elevator = has_elevator_widget.value
    house_type = house_type_widget.value
    PredictedYear = year_widget.value

    prediction, prediction_data, coordinates, reference_points = predict(address, living_space, room_count, property_age, has_basement, has_balcony, parking_lot_count, has_garden, has_elevator, house_type, PredictedYear)
    
    if prediction:
        display(widgets.Label(value=f'Prediction: {round(prediction * living_space, 1)}€')) #* living_space
        
        if coordinates is not None:
            m = folium.Map(location=[coordinates[0], coordinates[1]], zoom_start=14)
            folium.Marker([coordinates[0], coordinates[1]]).add_to(m)

            for _, row in reference_points.iterrows():
                folium.PolyLine([(coordinates[0], coordinates[1]), (row['Latitude'], row['Longitude'])], color="blue", weight=2.5, opacity=1).add_to(m)
            
            display(m)
        
        display(pd.DataFrame(prediction_data))
    else:
        display(widgets.Label(value='Cannot geocode address.'))
    
    display(address_widget, living_space_widget, room_count_widget, property_age_widget, has_basement_widget, has_balcony_widget, parking_lot_count_widget, has_garden_widget, has_elevator_widget, house_type_widget, year_widget, predict_button)


address_widget = widgets.Text(description='Address')
living_space_widget = widgets.FloatText(description='Living Space (sqm)')
room_count_widget = widgets.IntSlider(description='Room Count', min=1, max=10, value=2)
property_age_widget = widgets.IntSlider(description='Property Age', min=0, max=100, value=2)
has_basement_widget = widgets.ToggleButton(description='Has Basement', value=False)
has_balcony_widget = widgets.ToggleButton(description='Has Balcony', value=True)
parking_lot_count_widget = widgets.IntSlider(description='Parking Lot Count', min=0, max=10, value=0)
has_garden_widget = widgets.ToggleButton(description='Has Garden', value=False)
has_elevator_widget = widgets.ToggleButton(description='Has Elevator', value=True)
house_type_widget = widgets.Dropdown(description='House Type', options=['Apartment', 'Groundfloor','half basement', 'roof storey','maisonette','raised ground floor', 'terraced flat','penthouse', 'loft', 'other'], value='Apartment')
year_widget = widgets.IntSlider(description='PredictedYear', min=2022, max=2025, value=2023)

predict_button = widgets.Button(description='Predict')
predict_button.on_click(predict_button_clicked)

display(address_widget, living_space_widget, room_count_widget, property_age_widget, 
        has_basement_widget, has_balcony_widget, parking_lot_count_widget, has_garden_widget, 
        has_elevator_widget, house_type_widget, year_widget, predict_button)


1/1 [==============================] - 0s 23ms/step


Label(value='Prediction: 483.7€')

,roomCount,propertyAge,livingSpace,hasBasement,hasBalcony,parkingLotCount,hasGarden,hasElevator,houseType,count_nearby,mean_rent_per_sqm_nearby,std_rent_per_sqm_nearby,min_rent_per_sqm_nearby,max_rent_per_sqm_nearby,mean_hasBasement_nearby,mean_hasBalcony_nearby,mean_hasGarden_nearby,mean_hasElevator_nearby
0,2,19,40.0,False,True,4,False,True,1,20.0,11.91223,2.195419,9.458182,20.725714,0.5,0.75,0.05,0.45


Text(value='Feldstrasse 143, 22880 Wedel', description='Address')

FloatText(value=40.0, description='Living Space (sqm)')

...

ToggleButton(value=False, description='Has Garden')

ToggleButton(value=True, description='Has Elevator')

Dropdown(description='House Type', index=1, options=('Apartment', 'Groundfloor', 'half basement', 'roof storey…

IntSlider(value=2023, description='PredictedYear', max=2025, min=2022)

Button(description='Predict', style=ButtonStyle())